## What this notebook is
This notebook demonstrates how I trained Gemma-2 9b to obtain LB: 0.941. The inference code can be found [here](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora).
I used 4-bit quantized [Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit) uploaded by unsloth team as a base-model and added LoRA adapters and trained for 1 epoch.

## Result

I used `id % 5 == 0` as an evaluation set and used all the rest for training.

| subset | log loss |
| - | - |
| eval | 0.9371|
| LB | 0.941 |

## What is QLoRA fine-tuning?

In the conventional fine-tuning, weight ($\mathbf{W}$) is updated as follows:

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

where $L$ is a loss at this step and $\eta$ is a learning rate.

[LoRA](https://arxiv.org/abs/2106.09685) tries to approximate the $\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ by factorizing $\Delta \mathbf{W}$ into two (much) smaller matrices, $\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ and $\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ with $r \ll \text{min}(\text{d}, \text{k})$.

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

During training, only $\mathbf{A}$ and $\mathbf{B}$ are updated while freezing the original weights, meaning that only a fraction (e.g. <1%) of the original weights need to be updated during training. This way, we can reduce the GPU memory usage significantly during training while achieving equivalent performance to the usual (full) fine-tuning.

[QLoRA](https://arxiv.org/abs/2305.14314) pushes the efficiency further by quantizing LLM. For example, a 8B parameter model alone would take up 32GB of VRAM in 32-bit, whereas quantized 8-bit/4-bit 8B model only need 8GB/4GB respectively. 
Note that QLoRA only quantize LLM's weights in low precision (e.g. 8-bit) while the computation of forward/backward are done in higher precision (e.g. 16-bit) and LoRA adapter's weights are also kept in higher precision.

1 epoch using A6000 took ~15h in 4-bit while 8-bit took ~24h and the difference in log loss was not significant.

## Note
It takes prohivitively long time to run full training on kaggle kernel. I recommend to use external compute resource to run the full training.
This notebook uses only 100 samples for demo purpose, but everything else is same as my setup.

In [1]:
# gemma-2 is available from transformers>=4.42.3
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

Defaulting to user installation because normal site-packages is not writeable
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/12.0 MB


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Program Files\Python313\python.exe -m pip install --upgrade pip


In [ ]:
import os
import copy
import json
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

ModuleNotFoundError: No module named 'numpy'

### Configurations

In [3]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2  # global batch size is 8 
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

#### Training Arguments

In [4]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

#### LoRA config

In [5]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

### Instantiate the tokenizer & model

In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-15): 16 x Gemma2DecoderLayer(
            (self_attn): Gemma2SdpaAttention(
              (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
              (rotary_emb): Gemma2RotaryEmbedding()
            )
            (mlp): Gemma2MLP(
              (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=3584,

In [8]:
model.print_trainable_parameters()

trainable params: 7,891,456 || all params: 9,249,608,192 || trainable%: 0.0853


### Instantiate the dataset

In [ ]:
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
ds = ds.select(torch.arange(100))  # We only use the first 100 data for demo purpose

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        # Pre-process texts
        prompt = ["<prompt>: " + self.process_prompt(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_response(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_response(t) for t in batch["response_b"]]
        
        # Custom concatenation and truncation logic
        texts = []
        for p_parts, r_a_parts, r_b_parts, p, r_a, r_b in zip(batch["prompt"], batch["response_a"], batch["response_b"], prompt, response_a, response_b):
            full_text = p + r_a + r_b
            full_text_tokens = self.tokenizer.tokenize(full_text)
            total_length = len(full_text_tokens)
            
            if total_length <= self.max_length:
                texts.append(full_text)  # directly use the concatenated text
            else:
                # Calculate proportions
                len_p = len(self.tokenizer.tokenize(p))
                len_r_a = len(self.tokenizer.tokenize(r_a))
                len_r_b = len(self.tokenizer.tokenize(r_b))
                
                total_part_length = len_p + len_r_a + len_r_b
                proportion_p = len_p / total_part_length
                proportion_r_a = len_r_a / total_part_length
                proportion_r_b = len_r_b / total_part_length

                # Calculate tokens to keep per part
                tokens_to_keep_p = int(proportion_p * self.max_length)
                tokens_to_keep_r_a = int(proportion_r_a * self.max_length)
                tokens_to_keep_r_b = int(proportion_r_b * self.max_length)

                # function to be completed truncate_parts
                tokens_p = self.truncate_parts(p_parts,tokens_to_keep_p, "<prompt>: ", "prompt")
                tokens_r_a = self.truncate_parts(r_a_parts,tokens_to_keep_r_a, "\n\n<response_a>: ", "response")
                tokens_r_b = self.truncate_parts(r_b_parts,tokens_to_keep_r_b, "\n\n<response_b>: ", "response")
                
                texts.append(tokens_p + tokens_r_a + tokens_r_b)

        # Final tokenization step - ensure texts are in the right format (list of strings)
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True, padding=False)
        
        # Processing labels
        labels = [0 if a_win else 1 if b_win else 2 for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"])]
        return {**tokenized, "labels": labels}

    def truncate_parts(self, parts_text, max_length_for_segment, prefix, tag):
        # Evaluate the string representation of the list into an actual list
        parts = self.process_text(parts_text)

        # Tokenize each part separately and store tokens
        tokenized_parts = [self.tokenizer.tokenize(f"\n{tag} {i+1}: " + part) for i, part in enumerate(parts)]
        # Calculate the length of tokens for each part and the total length
        part_lengths = [len(tokens) for tokens in tokenized_parts]
        total_parts_length = sum(part_lengths)
        
        # Calculate the proportional maximum length for each part
        part_max_lengths = [(length / total_parts_length) * max_length_for_segment if total_parts_length > 0 else max_length_for_segment / len(parts) for length in part_lengths]
        # Truncate each part to its proportional length and decode
        truncated_parts = []
        for tokens, max_len in zip(tokenized_parts, part_max_lengths):
            truncated_tokens = tokens[:int(max_len)]
            token_ids = self.tokenizer.convert_tokens_to_ids(truncated_tokens) 
            truncated_text = self.tokenizer.decode(token_ids)
            truncated_parts.append(truncated_text)
        
        # Concatenate all truncated parts
        final_text = prefix + " ".join(truncated_parts) 
        return final_text

    def process_prompt(self, text: str) -> str:
        parts = self.process_text(text)
        return "".join(f"\nprompt {i+1}: {part}" for i, part in enumerate(parts))
    
    def process_response(self, text: str) -> str:
        parts = self.process_text(text)
        return "".join(f"\nresponse {i+1}: {part}" for i, part in enumerate(parts))

    @staticmethod
    def process_text(text: str):
        if isinstance(text, list):
            return text
        try:
            parsed = json.loads(text)
            if isinstance(parsed, list):
                return parsed
            return [str(parsed)]
        except:
            return [str(text)]
    

In [11]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

### Compute metrics

We'll compute the log-loss used in LB and accuracy as a auxiliary metric.

In [12]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

### Split

Here, train and eval is splitted according to their `id % 5`

In [13]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

In [14]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Acc,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,1.731400,1.320245,0.350000,1.320155,24.203000,0.826000,0.124000


TrainOutput(global_step=20, training_loss=1.7198352813720703, metrics={'train_runtime': 311.9326, 'train_samples_per_second': 0.256, 'train_steps_per_second': 0.064, 'total_flos': 2853279087925248.0, 'train_loss': 1.7198352813720703, 'epoch': 1.0})